In [2]:
import os
import matplotlib.pyplot as plt
import cv2
import numpy as np

gender_folder = r"C:\Users\agboo\Downloads\Gender_Classification"
gender_class = ["women","men"]

# we will use this to traverse our features and append them along with their classes
# we will make every picture 100 X 120  this is due to most pictures being selfies / and or full body
row = 120
col = 100
feature = []
def training():
    for gender in range(2):
        path = os.path.join(gender_folder, gender_class[gender])
        # creating flag we will classify with either 0 or 1
        flag = gender_class.index(gender_class[gender]) 
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path,img), cv2.IMREAD_COLOR)
                new_img = cv2.resize(img_array,(row,col))
                feature.append([new_img,flag])
            except:
                print(" Error: Corrupted File :( ")
training()
print(len(feature))

# import random so we can shuffle data
import random
random.shuffle(feature)

X = []
Y = []

for feat , ident in feature:
    X.append(feat)
    Y.append(ident)
X = np.array(X).reshape(-1,row,col,3) # features
Y = np.array(Y) # flags


## now we will save the data as pickle files to be loaded into the model
import pickle 
X_out = open("Feat.pickle","wb") # open file for data to then be dumped
pickle.dump(X, X_out)
X_out.close()

Y_out = open("Ident.pickle","wb") # the same is done for the flag list
pickle.dump(Y, Y_out)
Y_out.close()

#### the data will be loaded using rb (read binary when we start to construct the model




    
    

    

    

    
    
    

 Error: Corrupted File :( 
 Error: Corrupted File :( 
 Error: Corrupted File :( 
 Error: Corrupted File :( 
1581


In [52]:
import tensorflow as tf
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D , Dense , MaxPooling2D , Flatten , Activation , RandomZoom , Dropout , AveragePooling2D
from tensorflow.keras.callbacks import TensorBoard


import time
import pickle

X = pickle.load(open("Feat.pickle","rb"))
Y = pickle.load(open("Ident.pickle","rb"))

X = X/255.0 # this will normalize the feature array

# we will define this to display linked list
def pretty(root):
    while root:
        print(str(root.val) + "->",end="")
        root = root.next
    if root:
        print(root.val)
    else:
        print(None)
# we will have a conv node 
#hidden layers
# then a flatten layer
#dense layers
#output layer

# we will experiment with how we can improve performace by making our dense and fully connected layers bigger , as well as dense layers

 # globally create this model technically sequential but just with more steps not really making use of functional or subclassing like
# we wanted 


model = Sequential() # make model global

# window encompasses maxpoooling convolutions

# IMPORTANT "dimensions" and "nodes" are synoymous
def conv(dimensions , window , strider):
    model.add(Conv2D(dimensions ,(window,window) , input_shape = X.shape[1:]))
    model.add(Activation("relu"))# hardcoding relu within the convolutional layer because we are focused on feature extraction
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size = (window , window)))
    model.add(Activation("relu"))
    model.add(Dropout((0.3) , seed = 8))
    model.add(AveragePooling2D(pool_size = (window,window) , strides = (strider,strider)))
    model.add(Activation("relu"))
    model.add(Dropout((0.1) , seed = 9))
    

def hidden(dimensions , window , strider):
    model.add(Conv2D(dimensions,(window,window)))
    model.add(Activation("relu"))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size = (window,window)))
    model.add(Activation("relu"))
    model.add(Dropout((0.2) , seed = 2))
    model.add(AveragePooling2D(pool_size = (window,window) , strides = (strider,strider)))
    model.add(Activation("relu"))
    model.add(Dropout((0.1) , seed = 4))
    

# general hidden layers this will be made arbitarily long

def flatter():
    model.add(Flatten())

def dense(dimensions):
    model.add(Dense(dimensions))
    model.add(Activation("relu"))
    model.add(Activation("relu"))
    model.add(Dropout((0.2), noise_shape = None , seed = 4))
    model.add(Dense(dimensions))
    model.add(Activation("relu"))
    model.add(Activation("relu"))
    model.add(Dropout((0.1) , noise_shape = None , seed = 3))

# will make this arbitrarily big

def output():
    model.add(Dense(1))
    model.add(Activation("sigmoid")) # sigmoid because we have two classes

    model.compile(loss = "binary_crossentropy", optimizer = "adam" ,metrics = ["accuracy"]) 
## creating link classes

class ConvNode():
    def __init__(self,nodes, window,walk, val = 0 , next = None): # non default arguments must be before default unless you call half manually/non default
        self.val = val
        self.next = next
        conv(nodes,window,walk) # we can succesuflly invoke the function this way
class HiddenNode():
    def __init__(self,nodes,window,walk,val = 0, next = None):
        self.val = val
        self.next = next
        hidden(nodes,window,walk) # hidden call

class Flat_Node():
    def __init__(self,val = 0,next = None):
        self.val = val
        self.next = next
        flatter()

class Dense_Node():
    def __init__(self,nodes,val = 0,next = None):
        self.val = val
        self.next = None
        dense(nodes) #dense call
class Output_Node():
    def __init__(self,val = 0, next = None):
        self.val = val
        self.next = None
        output()

def create(nodes , window, walk, hidden_length, dense_length): # dimension , pooling size, length of hidden and length of dense
    count = 1
    cur = ConvNode(nodes,window,count)
    count += 1
    out = cur
    for i in range(hidden_length):
        cur.next = HiddenNode(nodes,window , count)
        cur = cur.next
        count += 1
    cur.next = Flat_Node(count)
    cur = cur.next
    count += 2
    for j in range(dense_length):
        cur.next = Dense_Node(nodes,count)
        cur = cur.next
        count += 1
    cur.next = Output_Node(count)
    cur = cur.next
    count += 1
    # display link
    return pretty(out)
  

create(18,2,2,1,1)
model.summary() # we will stop here 8/4/24 12:00 AM ; 51,823 Parameters ; Space : 202.43 KB


# now we will create a tensorboard
tensor = TensorBoard(log_dir = "Track_Django")

model.fit(X,Y, batch_size = 16 , epochs = 43 , validation_split = 0.2 , callbacks = [tensor])

model.save("Track_Django.keras") # will save this model and deploy it to our django web app




        
















        


    
        
        
        
        
   



0->0->3->5->6->None


Model: "sequential_43"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_102 (Conv2D)                  │ (None, 119, 99, 18)         │             234 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_529 (Activation)          │ (None, 119, 99, 18)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_530 (Activation)          │ (None, 119, 99, 18)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_98 (MaxPooling2D)      │ (None, 59, 49, 18)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_531 (Activation)          │ (None, 59, 49, 18)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_159 (Dropout)                │ (None, 59, 49, 18)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ average_pooling2d_63                 │ (None, 58, 48, 18)          │               0 │
│ (AveragePooling2D)                   │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_532 (Activation)          │ (None, 58, 48, 18)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_160 (Dropout)                │ (None, 58, 48, 18)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_103 (Conv2D)                  │ (None, 57, 47, 18)          │           1,314 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_533 (Activation)          │ (None, 57, 47, 18)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_534 (Activation)          │ (None, 57, 47, 18)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_99 (MaxPooling2D)      │ (None, 28, 23, 18)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_535 (Activation)          │ (None, 28, 23, 18)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_161 (Dropout)                │ (None, 28, 23, 18)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ average_pooling2d_64                 │ (None, 14, 11, 18)          │               0 │
│ (AveragePooling2D)                   │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_536 (Activation)          │ (None, 14, 11, 18)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_162 (Dropout)                │ (None, 14, 11, 18)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_36 (Flatten)                 │ (None, 2772)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_134 (Dense)                    │ (None, 18)                  │          49,914 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 51,823 (202.43 KB)

 Trainable params: 51,823 (202.43 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/43
79/79 ━━━━━━━━━━━━━━━━━━━━ 4s 34ms/step - accuracy: 0.5204 - loss: 0.6941 - val_accuracy: 0.4921 - val_loss: 0.6864
Epoch 2/43
79/79 ━━━━━━━━━━━━━━━━━━━━ 3s 35ms/step - accuracy: 0.5378 - loss: 0.6812 - val_accuracy: 0.5174 - val_loss: 0.6896
Epoch 3/43
79/79 ━━━━━━━━━━━━━━━━━━━━ 3s 35ms/step - accuracy: 0.5408 - loss: 0.6829 - val_accuracy: 0.5205 - val_loss: 0.6721
Epoch 4/43
79/79 ━━━━━━━━━━━━━━━━━━━━ 3s 32ms/step - accuracy: 0.5927 - loss: 0.6598 - val_accuracy: 0.6656 - val_loss: 0.6409
Epoch 5/43
79/79 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/step - accuracy: 0.6511 - loss: 0.6511 - val_accuracy: 0.6940 - val_loss: 0.6273
Epoch 6/43
79/79 ━━━━━━━━━━━━━━━━━━━━ 3s 32ms/step - accuracy: 0.6370 - loss: 0.6426 - val_accuracy: 0.6215 - val_loss: 0.6325
Epoch 7/43
79/79 ━━━━━━━━━━━━━━━━━━━━ 3s 32ms/step - accuracy: 0.6692 - loss: 0.6241 - val_accuracy: 0.6246 - val_loss: 0.6234
Epoch 8/43
79/79 ━━━━━━━━━━━━━━━━━━━━ 3s 32ms/step - accuracy: 0.6826 - loss: 0.6110 - val_accuracy: 0.7192 - v